In [ ]:
import os
import shutil
import numpy as np
import yaml

In [ ]:
with open("../config.yaml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)
    print("Read successful")
print(config)

In [ ]:
merged = config[0]['data']['merged']
fastp = config[0]['data']['fastp']
fastp_json = config[0]['data']['fastp_json']

In [ ]:
os.chdir(merged)
files = [f for f in os.listdir('.') if os.path.isfile(f)]
files.sort()
len(files)

In [ ]:
ids = []
for f in files:
    ids.append(f.split("_")[0])
ids = np.array(ids)
ids = np.unique(ids)
len(ids)

## Perform quality control with fastp

In [ ]:
for i in range(len(ids)):
    path = "".join([fastp, '/',ids[i],"_fastp"])
    r1 = "".join([ids[i], "_R1.fastq.gz"])
    r2 = "".join([ids[i], "_R2.fastq.gz"])
    out = "".join([path, "/", ids[i]])
    print(out)

    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)
        os.makedirs(path)

    os.system(f"fastp -i {r1} -I {r2} -h {out}.html -j {out}.json -o {out}\_R1\_fastp.fq.gz -O {out}\_R2\_fastp.fq.gz -w 16")

## Assemble results with multiqc

In [ ]:
os.chdir(fastp)
dirs = os.listdir(".")
len(dirs)

In [ ]:
os.system(f"find . -name *.json -exec cp {{}} {fastp_json} \;")

In [ ]:
os.chdir(fastp_json)
os.system("for filename in *.json; do mv $filename ${filename%.*}_fastp.json; done;")

In [ ]:
files = os.listdir(".")
len(files)

In [ ]:
os.system(f"multiqc {fastp_json} --filename multiqc_report_fastp -o {fastp_json}/.. -f --interactive")